# Spherical Fourier Neural Operators

A simple notebook to showcase spherical Fourier Neural Operators operating on MERRA2 data


## Preparation

In [1]:
%matplotlib ipympl
import torch, logging
import hydra, os, time
from typing import Any, Dict, List, Tuple, Type, Optional, Union, Sequence, Mapping
from fmod.base.util.dates import date_list
from fmod.base.util.config import cfg, start_date,  cfg2args, pp
from fmod.pipeline.merra2 import MERRA2Dataset
import torch, math
from fmod.base.util.config import cfg, cfg_date
from torch.utils.data import DataLoader
from fmod.base.util.grid import GridOps
from fmod.base.util.logging import lgm, exception_handled, log_timing
import torch.nn as nn


In [2]:
hydra.initialize(version_base=None, config_path="../config")
configure('merra2-sr')
lgm().set_level( logging.DEBUG )
loader_args = { k: cfg().model.get(k) for k in ['batch_size', 'num_workers', 'persistent_workers' ] }

# set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.cuda.set_device(device.index)

### Training data
to train our geometric FNOs, we require training data. To this end let us prepare a Dataloader which computes results on the fly:

In [6]:
dataset = MERRA2Dataset( train_dates=date_list( start_date( cfg().task ), cfg().task.max_steps ), vres="low" )
inp, tar = next(iter(dataset))
grid_shape = inp.shape[-2:]
lmax = grid_shape[0]

print(f"LOWRES SHAPES= {inp.shape}, {tar.shape}, (nlat, nlon)={grid_shape}, lmax={lmax}")

In [ ]:
dataset = MERRA2Dataset( train_dates=date_list( start_date( cfg().task ), cfg().task.max_steps ), vres="high" )
inp, tar = next(iter(dataset))
grid_shape = inp.shape[-2:]
lmax = math.ceil(grid_shape[0] / 3)

print(f"HRES SHAPES= {inp.shape}, {tar.shape}, (nlat, nlon)={grid_shape}, lmax={lmax}")